In [1]:
!pip install langchain llama-cpp-python PyMuPDF faiss-cpu sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 42.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.0 MB/s eta 0:00:00
  Us

In [2]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community
import fitz
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.1 MB/s eta 0:00:00


In [4]:
def extract_from_pdf(pdf_file):
  doc = fitz.open(pdf_file)
  text = ""
  for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text += page.get_text()
  return text

In [73]:
pdf_file = "./tcfd.pdf"
document_text = extract_from_pdf(pdf_file)

In [74]:
print(len(document_text))

138252


In [75]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [77]:
def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(document_text)

# Create the FAISS vector store
vector_store = FAISS.from_texts(chunks, embedding_model)

In [78]:
tokenizer = AutoTokenizer.from_pretrained("OuteAI/Lite-Mistral-150M-v2-Instruct")
model = AutoModelForCausalLM.from_pretrained("OuteAI/Lite-Mistral-150M-v2-Instruct")

In [79]:
retreiver = vector_store.as_retriever()

In [80]:
from langchain.chains import RetrievalQA

In [81]:
import transformers

In [82]:
from transformers import pipeline

In [83]:
hf_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

The model 'MistralForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [84]:
!pip install langchain-huggingface
from langchain_huggingface import HuggingFacePipeline


In [85]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [86]:
rag_pipeline=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retreiver)

In [87]:
def query_rag(question):
  response=rag_pipeline.run(question)
  return response

In [88]:
print(query_rag("How many climate action projects were supported in 2022?"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

tments in 2021 and 2022, the first two years of Climate Bank 
Roadmap implementation. These are presented in Table 2 below. 
Commitment indicators
2021 results
2022 results
Targets
1
Share of EIB finance dedicated to climate action and environmental 
sustainability
51% 
58% 
More than 
50% by 2025
2
Share of climate adaptation in EIB climate action finance
4.9%  
5.4%  
15% by 2025
3
Volume of climate action and environmental sustainability 
investment supported by the EIB Group*
€75 billion
€222 billion
€1 trillion by 
2030
* 
Reported cumulatively to reflect ten-year cumulative target. The figure in the 2022 results column therefore represents 
investment supported in 2021 and 2022.
Table 2: High-level commitments25 
25.	 Targets determined in the roadmap and subsequent EIB Climate Adaptation Plan.
39
METRICS AND TARGETS


In [ ]:
print(query_rag("Who is the CEO of Citi?"))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

s and Fraud Prevention and Chief Client Officer 
for Citi’s Personal Banking and Wealth Management 
(PBWM). Prior to joining Citi, Ms. Cole served as Head 
of Consumer and Small Business Banking Operations and 
Contact Centers at Wells Fargo, and before that, led Retail 
Products and Underwriting for Bank of America. 
•
Ms. Fraser joined Citi in 2004 and assumed her current 
position in March 2021. Previously, she served as CEO of 
Global Consumer Banking from October 2019 to 
December 2020. Before that, she served as CEO of Citi 
Latin America from June 2015 to October 2019. She held 
a number of other roles across the organization, including 
CEO of U.S. Consumer and Commercial Banking and 
CitiMortgage, CEO of Citi’s Global Private Bank and 
Global Head of Strategy and M&A.
•
Mr. Garg joined Citi in May 1988 and assumed h

In [72]:
print(query_rag("What are the challenges for Citi"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

ld otherwise allow Citi 
to be resolved in a way that protects systemically important 
functions without severe systemic disruption), or would not 
facilitate an orderly resolution of Citi under the U.S. 
Bankruptcy Code, and Citi fails to resubmit a resolution plan 
that remedies any identified deficiencies, Citi could be 
subjected to more stringent capital, leverage or liquidity 
requirements, or restrictions on its growth, activities or 
operations. If within two years from the imposition of any 
such requirements or restrictions Citi has still not remediated 
any identified deficiencies, then Citi could eventually be 
required to divest certain assets or operations. Any such 
restrictions or actions would negatively impact Citi’s 
reputation, market and investor perception, operations and 
strategy.
Citi’s Performance a